<i>STATUS: Draft<i>

In [293]:
import numpy as np
import sympy as sp
from IPython.display import HTML, IFrame
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.patches import Rectangle
from IPython.display import Image
import sys
import HTM_Code as hc
import pandas as pd

In the last notebook, we built our first encoder. Admittedly, it was very simple. It just allows us to create integers and provide them to, and provide back indice location. It provided a little extra functionality so it could track previous, and comparison. And we established some rules that we would like to see in place. 

We also discussed what similiarity metrics in such an environment might look like, based on the rules. Its important to emphasie that we have really conflated the idea of semantic meaning, with distance, which is problematic, and we will need to take a deeper look at what it means to really encode something semantically. 

Finally, the Encoder Class that was introduced in the last notebook is also included in the HTM_Code.py file for us to use


<div style="background:#99ddff; color:black; padding: 10px">
<b>Add to these notes:</b>

I want to keep these notes in HTML so I don't have to host on a server, but a great exceriise is to use the ipython widgets to experience this like it happens in the video. 
</div>

In [294]:
IFrame("https://www.youtube.com/embed/PTYlge2K1G8", width=600, height=300)

This episode is about how we might build Date and Time Encoder, but the way I like to think of this is . But the idea behind this is really that join multiple encoders togehter. For example, I might be interested in an encoder that takes 365 values, one for each day of the year, or just 2 values (one for whether it is weekend or not the weekend), 4 values (telling me which of the seasons of the year it is), or a minute encoder (telling which minute of the day). 

Under the hood, these are each no different from the encoders that we were already looking at. Each value that would stored is simply and SDR with active bits and an SDR size. 

I could join each of these encoders together (by simply concatentating the arrays), and I would in principle have an encoder that tracks the meaning of theach these, and I could feed into it a date and time, and see thier differen they are. 

To explore this further, let's build another class: a multi enncoder

In [295]:
class Encoder:
    def __init__(self, bit_space_size = None,
                number_of_bits_used_to_encode_value = None,
                min_val = None,
                max_val = None,
                is_randomly_distributed = None,
                clip_values_outside_range = None):

        self.bit_space_size = bit_space_size
        self.number_of_bits_used_to_encode_value = number_of_bits_used_to_encode_value
        self.clip_values_outside_range = clip_values_outside_range
        self.is_periodic = False
        self.is_randomly_distributed = is_randomly_distributed

        self.resolution = 1
        self.uniqueness = 1
        self.min_value_to_encode = min_val
        self.max_value_to_encode = max_val
        self.max_bit_space_value = bit_space_size
        self.min_bit_space_value = 0
        self.encoded_values = []
        self.encoded_values_bit_locations = []
        self.offset_for_array_indice = 1
        
        self.bucket_capacity = self.compute_bucket_capacity(self.bit_space_size, self.number_of_bits_used_to_encode_value)
        
        if self.is_randomly_distributed:
            self.initial_encoding = np.array(hc.create_randomised_sdr(self.bit_space_size, self.number_of_bits_used_to_encode_value))

            self.encoded_values_and_bit_locations = {str(self.min_value_to_encode):self.initial_encoding}
            self.encoded_values.append(self.min_value_to_encode)
            self.encoded_values_bit_locations.append(np.array(self.initial_encoding))
        
    def get_summary(self):
        print("----------------- SUMMARY -------------------------")
        print("|L3| Bit Space Size: ", self.bit_space_size)
        print("|L4| Number of bits to be used when encoding each value:", self.number_of_bits_used_to_encode_value)
        print("|L5| Range of values that can be encoded: From ", self.min_value_to_encode, ' to ', self.max_value_to_encode)
        print("|L6| Number of buckets available in bit space:", float(self.bucket_capacity))
        print("|L1| Encode periodically: ", self.is_periodic)
        print("|L1| Values are encoded as are randomly distributed arrays: ", self.is_randomly_distributed)
        print("|L1| Resolution: ", self.resolution)
        print("|L1| Unique active bits per bucket: ", self.uniqueness)
        print("|L2| Values outside range will to be clipped: ",self.clip_values_outside_range)
        print("|L7| Encoded values bit locations:\n ", self.encoded_values_bit_locations)
        print("|L8| Encoded values", self.encoded_values)
        print("----------------------------------------------------")

        
    def compute_bucket_capacity(self, n, w):
        if self.is_randomly_distributed:
            return(sp.binomial(self.bit_space_size, self.number_of_bits_used_to_encode_value))
        else:
            return(n - w + 1)

    def create_buckets_for_randomly_encoded_values(self, iterations_needed):
        
        for i in range(0, iterations_needed):
            random_bit_index_to_move = np.random.randint(0, self.number_of_bits_used_to_encode_value, 1)[0]
            random_direction_to_move = np.random.randint(0, 2, 1)

            next_sdr = self.encoded_values_bit_locations[-1].copy()
            value = next_sdr[random_bit_index_to_move]
            
            if random_direction_to_move == 1:
                value = next_sdr[random_bit_index_to_move] + 1
            else: 
                value = next_sdr[random_bit_index_to_move] - 1
                
            if value > self.max_bit_space_value:
                value = value - 2
            elif value < 0:
                value = value + 2

            next_sdr[random_bit_index_to_move] = value

            self.encoded_values_bit_locations.append(next_sdr.copy())
            self.encoded_values.append(np.array(self.encoded_values[-1] + 1))
            self.encoded_values_and_bit_locations[str(self.encoded_values[-1])] = next_sdr.copy()
  

    def encode_value_in_bit_space(self, value_choice):
        print("\nEncoding the value ->", value_choice)
        unclipped_value = value_choice
        if self.clip_values_outside_range:
            if value_choice < self.min_value_to_encode or value_choice > self.max_value_to_encode:
                if value_choice < self.min_value_to_encode:
                    value_choice = self.min_value_to_encode
                else:
                    value_choice = self.max_value_to_encode
                print("The value of: ", unclipped_value, "has been clipped to ->", value_choice)
            elif value_choice > self.min_value_to_encode or value_choice < self.max_value_to_encode:
                pass
        else:
            print("Not a valid choice, ", value_choice, " is outside encoder range")
            return

        
        if self.is_randomly_distributed:
            if (value_choice < self.encoded_values[-1]):
                print("There is a bucket already created for the value", value_choice, "-> ", self.encoded_values_and_bit_locations[str(value_choice)])
                if unclipped_value < self.min_value_to_encode or unclipped_value > self.max_value_to_encode:
                    print("This bucket will be used to encode", unclipped_value)
                    self.encoded_values_and_bit_locations[str(unclipped_value)] = self.encoded_values_and_bit_locations[str(value_choice)]
                return
            
            buckets_needed_to_encode_value = value_choice - self.encoded_values[-1]
            print("Current number of buckets: " , len(self.encoded_values))
            print("Value held in first bucket: ", self.min_value_to_encode)
            print("Number of additional buckets required to accomodate the value choice of", value_choice, ": ", buckets_needed_to_encode_value)
            self.create_buckets_for_randomly_encoded_values(buckets_needed_to_encode_value)
            self.encoded_values_and_bit_locations[str(unclipped_value)] = self.encoded_values_and_bit_locations[str(value_choice)]
        
        else:
            window = [value_choice, value_choice + self.number_of_bits_used_to_encode_value]
            all_values = np.arange(window[0], window[1])
            self.encoded_values_bit_locations.append(all_values)
            self.encoded_values.append(value_choice)
 

In [296]:
# bit_space_size_choice = 64
# number_of_bits_used_to_encode_value_choice = 8

# e1 = Encoder(bit_space_size = bit_space_size_choice,
#                 number_of_bits_used_to_encode_value = number_of_bits_used_to_encode_value_choice,
#                 min_val = 0,
#                 max_val = 1,
#             is_randomly_distributed = False,
#             clip_values_outside_range = False)

To explore this, let's use some data. There is some really interesting data that will turn up in episode's 7 and 8 in the context of the Spatial Pooler, that has some interesting info, but for now, let's use Baltimore data. THis data has nice coverage across a number of data points, descriptive names, some categorical variables, the footprint isn't too big but it gives us a nice sample of 96k records

Information available <a href="https://data.baltimorecity.gov/datasets/baltimore::part1-crime-2015-to-2016/about">https://data.baltimorecity.gov/datasets/baltimore::part1-crime-2015-to-2016/about</a>


In [348]:
# df = pd.read_csv("./data/data-society-nyc-crime-data/data/nypd_complaint_data_current_ytd.csv")
# df = pd.read_csv("./data/NYCCrimeData.csv")
# df.info()
df = pd.read_csv("./data/Part1_Crime_2015_to__2016.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96976 entries, 0 to 96975
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowID            96976 non-null  int64  
 1   CrimeDateTime    96976 non-null  object 
 2   CrimeTime        96969 non-null  object 
 3   CrimeCode        96976 non-null  object 
 4   Location         96399 non-null  object 
 5   Description      96976 non-null  object 
 6   Inside_Outside   94136 non-null  object 
 7   Weapon           19832 non-null  object 
 8   Post             96797 non-null  object 
 9   District         96797 non-null  object 
 10  Neighborhood     96790 non-null  object 
 11  Latitude         96921 non-null  float64
 12  Longitude        96921 non-null  float64
 13  GeoLocation      96976 non-null  object 
 14  Premise          94007 non-null  object 
 15  VRIName          11491 non-null  object 
 16  Total_Incidents  96976 non-null  int64  
 17  HashedRecord

The first thing I want to do is create some more readable names and fix some formatting so make it easier to deal with. I am also going to add some derived data, so we can see day, season which will show us more about our encoder

In [371]:
df = pd.read_csv("./data/Part1_Crime_2015_to__2016.csv")
df.CrimeDateTime = df.CrimeDateTime.str.slice(0, -8)
df.CrimeDateTime= pd.to_datetime(df.CrimeDateTime)
df['weekdayCodeWhenEventReported'] = [d.weekday() for d in df.CrimeDateTime]
df['monthCodeWhenEventReported'] = df['CrimeDateTime'].dt.month
df['seasonCodeWhenEventReported'] = (df['CrimeDateTime'].dt.month - 1) % 4

#df.weekdayCodeWhenEventReported = int(df.weekdayCodeWhenEventReported)
#df['weekdayNameWhenEventReported'] = [calendar.day_name[d.weekday()] for d in df.weekdayCodeWhenEventReported]

In [372]:
df[0:3].transpose()

,0,1,2
RowID,183321,183322,183323
CrimeDateTime,2016-12-31 00:00:00,2016-12-31 00:00:00,2016-12-31 00:00:00
CrimeTime,3:45:00,4:18:00,0:15:00
CrimeCode,3AF,3AF,4B
Location,300 YALE AVE,200 S BEECHFIELD AVE,2400 CHRISTIAN ST
Description,ROBBERY - STREET,ROBBERY - STREET,AGG. ASSAULT
Inside_Outside,O,O,I
Weapon,FIREARM,FIREARM,KNIFE
Post,824,822,834
District,SOUTHWEST,SOUTHWEST,SOUTHWEST


In [373]:
pd.value_counts(df.seasonCodeWhenEventReported)

3.0    24878
0.0    24444
2.0    24198
1.0    23449
Name: seasonCodeWhenEventReported, dtype: int64

In [341]:
df

,RowID,CrimeDateTime,CrimeTime,CrimeCode,Location,Description,Inside_Outside,Weapon,Post,District,Neighborhood,Latitude,Longitude,GeoLocation,Premise,VRIName,Total_Incidents,HashedRecord,ObjectId
0,183321,2016-12-31,3:45:00,3AF,300 YALE AVE,ROBBERY - STREET,O,FIREARM,824,SOUTHWEST,IRVINGTON,39.2816,-76.6857,"(39.2816,-76.6857)",STREET,NaN,1,0x53BB117C2ECA46991C79D80C3EF06BABAF99E50D30C3...,1
1,183322,2016-12-31,4:18:00,3AF,200 S BEECHFIELD AVE,ROBBERY - STREET,O,FIREARM,822,SOUTHWEST,TREMONT,39.2821,-76.6930,"(39.2821,-76.693)",STREET,NaN,1,0xAA5486B82835B0CCFF8B4BD262B85A6199D145C7E640...,2
2,183323,2016-12-31,0:15:00,4B,2400 CHRISTIAN ST,AGG. ASSAULT,I,KNIFE,834,SOUTHWEST,MILLHILL,39.2804,-76.6539,"(39.2804,-76.6539)",HOSP/NURS. HOME,Tri-District,1,0xE0056207362E7E877801A09B43F8734F1DEBA6B78A51...,3
3,183324,2016-12-31,8:50:00,4A,4600 REISTERSTOWN RD,AGG. ASSAULT,I,FIREARM,613,NORTHWEST,CENTRAL PARK HEIGHTS,39.3407,-76.6726,"(39.3407,-76.6726)",GROCERY/CONFECTION,NaN,1,0x0DA4D6C840CD34535EED02C881EF9937320869B4AD87...,4
4,183325,2016-12-31,19:00:00,7A,3200 BAKER ST,AUTO THEFT,O,NaN,813,SOUTHWEST,ROSEMONT,39.3058,-76.6715,"(39.3058,-76.6715)",STREET,NaN,1,0xF4EE13FA98047B9538A501BBBFC41CCDB9F219B3DF41...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96971,280292,2015-01-01,11:51:00,8FO,2500 W BELVEDERE AVE,ARSON,I,FIRE,532,NORTHERN,LEVINDALE,39.3528,-76.6668,"(39.3528,-76.6668)",HOSP/NURS. HOME,NaN,1,0xFEA515D7B825FE1A89561E0A5AD817E941DF3DA24F54...,96972
96972,280293,2015-01-01,10:00:00,5D,3900 WABASH AVE,BURGLARY,I,NaN,625,NORTHWEST,EAST ARLINGTON,39.3291,-76.6657,"(39.3291,-76.6657)",GARAGE ON PRIV. PROP,NaN,1,0x3071510616731622F63CC3B5CE3BB0626CAB7B8EA5BC...,96973
96973,280294,2015-01-01,1:00:00,6E,1200 N BENTALOU ST,LARCENY,O,NaN,723,WESTERN,BRIDGEVIEW/GREENLAWN,39.3018,-76.6537,"(39.3018,-76.6537)",STREET,NaN,1,0xBFE7E1F0386D0E006309E5F5D2272D41950875E7DB44...,96974
96974,280295,2015-01-01,16:00:00,6D,700 W HAMBURG ST,LARCENY FROM AUTO,O,NaN,932,SOUTHERN,WASHINGTON VILLAGE,39.2808,-76.6276,"(39.2808,-76.6276)",STREET,NaN,1,0xA3DF595FF8527C93D89CF10154C3DBC00810E218A1E7...,96975


In [308]:
#df = pd.read_csv("./data/data-society-nyc-crime-data/data/nypd_complaint_data_current_ytd.csv")


df = df.rename(columns={"cmplnt_num": "uniqueID",
                        "cmplnt_fr_tm": "timeOfDayOfOccurence",
                        "rpt_dt":"dateEventReported",
                        "ky_cd":"offenseClassificationCode",
                        "ofns_desc":"offenseClassificationDescription",
                        "pd_cd": "internalClassificationCode",
                        "pd_desc":"internalClassificationDescription",
                        "crm_atpt_cptd_cd": "crimeSuccessStatus",
                        "law_cat_cd": "offenseLevel",
                        "juris_desc": "jurisdictionCodeDescription",
                        "boro_nm":"nameOfBouough",
                        "addr_pct_cd":  "precinctWhereOccured",
                        "loc_of_occur_desc":"positionLocationOfOccurence",
                        "prem_typ_desc":"premisesDescription",
                        "parks_nm":"greenspaceNameIfApplicable",
                        "hadevelopt":"housingDevelopmentNameIfApplicable" })


df.dateEventReported = pd.to_datetime(df.dateEventReported)

df.timeOfDayOfOccurence = pd.to_datetime(df.timeOfDayOfOccurence)
df.timeOfDayOfOccurence= df.timeOfDayOfOccurence.dt.time

df['weekdayCodeWhenEventReported'] = [d.weekday() for d in df.dateEventReported]
df['weekdayNameWhenEventReported'] = [calendar.day_name[d.weekday()] for d in df.dateEventReported]
df['monthCodeWhenEventReported'] = df['dateEventReported'].dt.month
df['seasonCodeWhenEventReported'] = (df['dateEventReported'].dt.month - 1) % 3

df = df.drop('x_coord_cd', axis=1)
df = df.drop('y_coord_cd', axis=1)

df = df.drop('cmplnt_fr_dt', axis=1)
df = df.drop('cmplnt_to_dt', axis=1)
df = df.drop('cmplnt_to_tm', axis=1)

df = df.drop('lat_lon', axis=1)
df = df.drop('location', axis=1)


Let's check out the first few records:

In [309]:
df[1:5].transpose()

,1,2,3,4
Unnamed: 0,1,2,3,4
uniqueID,294332956,852981427,369976063,117213771
timeOfDayOfOccurence,23:16:00,23:00:00,23:00:00,23:00:00
dateEventReported,2016-09-30 00:00:00,2016-09-30 00:00:00,2016-09-30 00:00:00,2016-09-30 00:00:00
offenseClassificationCode,344,235,118,578
offenseClassificationDescription,ASSAULT 3 & RELATED OFFENSES,DANGEROUS DRUGS,DANGEROUS WEAPONS,HARRASSMENT 2
internalClassificationCode,101.0,567.0,793.0,637.0
internalClassificationDescription,ASSAULT 3,"MARIJUANA, POSSESSION 4 & 5",WEAPONS POSSESSION 3,"HARASSMENT,SUBD 1,CIVILIAN"
crimeSuccessStatus,COMPLETED,COMPLETED,COMPLETED,COMPLETED
offenseLevel,MISDEMEANOR,MISDEMEANOR,FELONY,VIOLATION


In [310]:
pd.value_counts(df.monthCodeWhenEventReported)

9    39850
8    10150
Name: monthCodeWhenEventReported, dtype: int64

There is some interesting data here, some some scalar type values, and 

In [204]:
#df.info()

In [105]:
df = pd.read_csv("./data/gymdata.csv", header=1)
df = df.rename(columns={"datetime": "dateTime", "float": "powerConsumption"})
df = df.iloc[1:]
df.dateTime = pd.to_datetime(df.dateTime, format="%m/%d/%y %H:%M")
df['date'] = [d.date() for d in df.dateTime]
df['timeOfDay'] = [d.time() for d in df.dateTime]
df['dayCode'] = [d.weekday() for d in df.date]
df['monthCode'] = df['dateTime'].dt.month
#df['monthCode'] = (df['dateTime'].dt.month - 1) % 3
df['isWeekend'] = np.where(df.dayCode > 4, True, False)
df['dayName'] = [calendar.day_name[d.weekday()] for d in df.date]
# df = df.drop('date_time', axis=1)
#df = df.drop('weekday', axis=1)



In [106]:
pd.value_counts(df.monthCode)

8     744
10    744
12    743
9     720
7     720
11    720
Name: monthCode, dtype: int64

In [101]:
df[df.isWeekend == True]

,dateTime,powerConsumption,date,timeOfDay,dayCode,monthCode,isWeekend,dayName
25,2010-07-03 00:00:00,22.5,2010-07-03,00:00:00,5,0,True,Saturday
26,2010-07-03 01:00:00,22.3,2010-07-03,01:00:00,5,0,True,Saturday
27,2010-07-03 02:00:00,6.0,2010-07-03,02:00:00,5,0,True,Saturday
28,2010-07-03 03:00:00,4.7,2010-07-03,03:00:00,5,0,True,Saturday
29,2010-07-03 04:00:00,4.4,2010-07-03,04:00:00,5,0,True,Saturday
...,...,...,...,...,...,...,...,...
4268,2010-12-26 19:00:00,5.5,2010-12-26,19:00:00,6,2,True,Sunday
4269,2010-12-26 20:00:00,5.5,2010-12-26,20:00:00,6,2,True,Sunday
4270,2010-12-26 21:00:00,5.2,2010-12-26,21:00:00,6,2,True,Sunday
4271,2010-12-26 22:00:00,4.9,2010-12-26,22:00:00,6,2,True,Sunday


In [51]:
from datetime import date
import calendar
my_date = date.today()
print(my_date.weekday())
calendar.day_name[my_date.weekday()]  #'Wednesday'

2


'Wednesday'

Now onto next episode of HTM School

In [6]:
class MultiEncoder:
    def __init__(self):
        self.encoders = []
        self.bit_space_size = None
    def add_encoder(self, encoder):
        self.encoders.append(encoder)
    def join_encoders(self):
        pass

So this is a really powerful idea. We can think about multiple time series unfolding, date time components. Or we could think of symbolic music, track key changes

In [ ]:
def compute_union_and_overlap(SDR1_on_bits, SDR2_on_bits):
    union = list(set(SDR1_on_bits).union(SDR2_on_bits))
    overlap = list(set(SDR1_on_bits).intersection(SDR2_on_bits))
    
    return({"union": union, "overlap": overlap})



This is starting to seem more like we saw in the early notebooks, we can how semantic similairty is affected by noise and subsampling

There is alot more we can do with encoders. Delta encoder, log encoder. A geospatial encoder particularly interesting, enconding values on a sphere, what other geometrical shapes, opens us up to diffent types of geometry adn topology also

Let's look at more encoders: 
https://numenta.com/assets/pdf/biological-and-machine-intelligence/BaMI-Encoders.pdf

IMportant to capture semantics properly

4 principles recall..... eg consider two numbers should be semantically similiar 

Note for encoders, we can encode anything we can put in a relationship of 

we need encoders to have to to incorporate noise and subsamplinig 

Encoding Daa for HTM systems - Purdy


Date Time Encoder

 
